# Connectivity - distance between MPAs 3.1

### Import libraries and set up environment

In [4]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [5]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# make a custom projection called Azimuthal Equidistant (world)_carib:

# Alter Azimuthal Equidistant (world) such that:
# central meridian is -80 and latitude of origin is 20



In [ ]:
# project the dissolved MPA layer to the custom projection

in_feature = "WDPA_MPA_dis"
out_feature = "WDPA_MPA_dis_Azi"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"


arcpy.management.Project(
    in_dataset= in_feature,
    out_dataset= out_feature,
    out_coor_system='PROJCS["Azimuthal Equidistant (world)_carib",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-80.0],PARAMETER["Latitude_Of_Origin",20.0],UNIT["Meter",1.0]]',
    transform_method=None,
    in_coor_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
    preserve_shape="NO_PRESERVE_SHAPE",
    max_deviation=None,
    vertical="NO_VERTICAL"
)

In [ ]:
# Calculate the area of the MPA footprint in Mollweide

in_feature = "WDPA_MPA_dis_Azi"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.management.CalculateGeometryAttributes(
    in_features= in_feature,
    geometry_property="area AREA",
    length_unit="",
    area_unit="SQUARE_KILOMETERS",
    coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

In [3]:
# Remove bits that are less than .24 km2

in_feature = "WDPA_MPA_dis_Azi"
out_feature = "WDPA_MPA_dis_Azi_big"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def select_big_repair():
    # make a layer of polygons with an area greater than .24 km2    
    arcpy.conversion.ExportFeatures(
        in_features= in_feature,
        out_features= out_feature,
        where_clause="area >= 0.24",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=None,
        sort_field=None
    )
    
    # repair geometry
    arcpy.management.RepairGeometry(
        in_features= out_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
    
select_big_repair()


In [25]:
MPA_footprint_count = int(arcpy.GetCount_management("WDPA_MPA_dis_Azi_big").getOutput(0))
print(f"Number of MPA footprints: {MPA_footprint_count}")

Number of MPA footprints: 677


In [ ]:
# Buffer the layer by 10 km

in_feature = "WDPA_MPA_dis_Azi_big"
out_feature = "WDPA_MPA_dis_Azi_big_buff"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseBuffer(
    in_features= in_feature,
    out_feature_class= out_feature,
    buffer_distance_or_field="10 Kilometers",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR",
    max_deviation="0 Meters"
)

In [ ]:
# Intersect the buffered MPAs

in_feature = "WDPA_MPA_dis_Azi_big_buff"
out_feature = "WDPA_MPA_dis_Azi_big_buff_int"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseIntersect(
    in_features= in_feature,
    out_feature_class=out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

In [20]:
# Make layers of the connected and not connected MPAs by re-matching the intersecting buffers with their corresponding MPA footprints

in_feature = "WDPA_MPA_dis_Azi_big_buff"
select_feature = "WDPA_MPA_dis_Azi_big_buff_int"
MPAs = "WDPA_MPA_dis_Azi_big"
out_feature = "WDPA_MPA_dis_Azi_big_conn"
out_feature_not_connected = "WDPA_MPA_dis_Azi_big_NotConn"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def connected_mpas():
    
    #select the intersecting buffers
    selection1 = arcpy.management.SelectLayerByLocation(
        in_layer= in_feature,
        overlap_type="INTERSECT",
        select_features= select_feature,
        search_distance=None,
        selection_type="NEW_SELECTION",
        invert_spatial_relationship="NOT_INVERT"
    )
    
    selected_count = int(arcpy.GetCount_management(selection1).getOutput(0))
    print(f"Number of selected features in {selection1}: {selected_count}")

    # Convert those back to their corresponing MPA footprints by selecting the MPAs that are completely within the selected buffers
    selection2 = arcpy.management.SelectLayerByLocation(
        in_layer= MPAs,
        overlap_type="COMPLETELY_WITHIN",
        select_features= selection1,
        search_distance=None,
        selection_type="ADD_TO_SELECTION",
        invert_spatial_relationship="NOT_INVERT"
    )
    
    selected_count = int(arcpy.GetCount_management(selection2).getOutput(0))
    print(f"Number of selected features in {selection2}: {selected_count}")
    
    # make a new layer of the connected MPAs
    arcpy.conversion.FeatureClassToFeatureClass(
        in_features=selection2,
        out_path=output_gdb,
        out_name=out_feature,
        where_clause="",
    )
    
    #select the inverse of selection2 to make a layer of not connected MPAs
    selection2_inverse = arcpy.management.SelectLayerByLocation(
        in_layer= MPAs,
        overlap_type="COMPLETELY_WITHIN",
        select_features= selection1,
        search_distance=None,
        selection_type="ADD_TO_SELECTION",
        invert_spatial_relationship="INVERT"
    )
    
    selected_count = int(arcpy.GetCount_management(selection2_inverse).getOutput(0))
    print(f"Number of selected features in {selection2_inverse}: {selected_count}")
    
    # make a new layer of the connected MPAs
    arcpy.conversion.FeatureClassToFeatureClass(
        in_features=selection2_inverse,
        out_path=output_gdb,
        out_name=out_feature_not_connected,
        where_clause="",
    )
    

connected_mpas()

Number of selected features in WDPA_MPA_dis_Azi_big_buff_La14: 603
Number of selected features in WDPA_MPA_dis_Azi_big_Layer8: 603
Number of selected features in WDPA_MPA_dis_Azi_big_Layer9: 74


In [31]:
# Create a Pandas DataFrame to store the selection counts
data = {
    "Total number of MPA footprints": [int(arcpy.GetCount_management("WDPA_MPA_dis_Azi_big").getOutput(0))],
    "Number of connected MPA footprints": [int(arcpy.GetCount_management("WDPA_MPA_dis_Azi_big_conn").getOutput(0))],
    "Number of not connected MPA footprints": [int(arcpy.GetCount_management("WDPA_MPA_dis_Azi_big_NotConn").getOutput(0))]
}
df = pd.DataFrame(data)
df.head()



,Total number of MPA footprints,Number of connected MPA footprints,Number of not connected MPA footprints
0,677,603,74


In [32]:
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Connectivity"
table = df
output_name = "Connectivity3_1_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))